![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/21.Gender_Classifier.ipynb)

# Gender Classifier 

**Gender Classifier** detects the gender of the patient in the clinical document. 
It can  classify the documents into `Female`, `Male` and `Unknown`.

We have two different models: 

-'**Classifierdl_gender_sbert**' (more accurate, works with licensed `sbiobert_base_cased_mli`)

-'**Classifierdl_gender_biobert**' (works with `biobert_pubmed_base_cased`)

The models are trained on more than four thousands clinical documents (radiology reports, pathology reports, clinical visits etc.) which were annotated internally.


## Colab Setup

In [ ]:
import json

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

license_keys.keys()

In [3]:
license_keys['JSL_VERSION']

'2.7.2'

In [ ]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

secret = license_keys['SECRET']

os.environ['SPARK_NLP_LICENSE'] = license_keys['SPARK_NLP_LICENSE']
os.environ['AWS_ACCESS_KEY_ID']= license_keys['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = license_keys['AWS_SECRET_ACCESS_KEY']
version = license_keys['PUBLIC_VERSION']
jsl_version = license_keys['JSL_VERSION']

! pip install --ignore-installed -q pyspark==2.4.4

! python -m pip install --upgrade spark-nlp-jsl==$jsl_version  --extra-index-url https://pypi.johnsnowlabs.com/$secret

! pip install --ignore-installed -q spark-nlp==$version

import sparknlp

print (sparknlp.version())

import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession


from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl

spark = sparknlp_jsl.start(secret)

In [6]:
spark

In [ ]:
# if you want to start the session with custom params as in start function above
def start(secret):
    builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:"+version) \
        .config("spark.jars", "https://pypi.johnsnowlabs.com/"+secret+"/spark-nlp-jsl-"+jsl_version+".jar")
      
    return builder.getOrCreate()

#spark = start(secret)

In [5]:
import sparknlp

print (sparknlp.version())

print (sparknlp_jsl.version())

2.6.5
2.7.2




# Gender Classifier Pipeline with **sbert**

In [42]:

document = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

sbert_embedder = BertSentenceEmbeddings().pretrained("sbiobert_base_cased_mli", 'en', 'clinical/models')\
      .setInputCols(["document"])\
      .setOutputCol("sentence_embeddings")\
      .setMaxSentenceLength(512)

gender_classifier = ClassifierDLModel.pretrained( 'classifierdl_gender_sbert', 'en', 'clinical/models') \
      .setInputCols(["document", "sentence_embeddings"]) \
      .setOutputCol("class")    

gender_pred_pipeline_sbert = Pipeline(stages = [ document, 
                                          sbert_embedder, 
                                          gender_classifier   
                                          ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model_sbert = gender_pred_pipeline_sbert.fit(empty_data)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
classifierdl_gender_sbert download started this may take some time.
Approximate size to download 22.4 MB
[OK!]
sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 321.1 KB
[OK!]


In [147]:
text ="""social history: shows that  does not smoke cigarettes or drink alcohol,lives in a nursing home.family history: shows a family history of breast cancer."""

gender_pipeline_sbert = LightPipeline(model_sbert)

result = gender_pipeline_sbert.annotate(text)

result['class'][0]


'Female'

### Sample Clinical Notes

In [148]:
text1 = '''social history: shows that  does not smoke cigarettes or drink alcohol,lives in a nursing home.
family history: shows a family history of breast cancer.'''

result = gender_pipeline_sbert.annotate(text1)

result['class'][0]

'Female'

In [149]:
text2 = '''The patient is a 48- year-old, with severe mitral stenosis diagnosed by echocardiography, moderate
 aortic insufficiency and moderate to severe pulmonary hypertension who is being evaluated as a part of a preoperative 
 workup for mitral and possible aortic valve repair or replacement.'''

result = gender_pipeline_sbert.annotate(text2)

result['class'][0]

'Unknown'

In [150]:

text3 = '''HISTORY: The patient is a 57-year-old XX, who I initially saw in the office on 12/27/07, as a referral from the Tomball Breast Center.
On 12/21/07, the patient underwent image-guided needle core biopsy of a 1.5 cm lesion at the 7 o'clock position of the left breast (inferomedial). 
The biopsy returned showing infiltrating ductal carcinoma high histologic grade.
The patient stated that xx had recently felt and her physician had felt a palpable mass in that area prior to her breast imaging.'''

result = gender_pipeline_sbert.annotate(text3)

result['class'][0]

'Female'

In [151]:

text4 = '''The patient states that xx has been overweight for approximately 35 years and has tried multiple weight loss modalities in 
the past including Weight Watchers, NutriSystem, Jenny Craig, TOPS, cabbage diet, grape fruit diet, Slim-Fast, Richard Simmons,
as well as over-the-counter  measures without any long-term sustainable weight loss.
At the time of presentation to the practice, xx is 5 feet 6 inches tall with a weight of 285.4 pounds and a body mass index of 46.
xx has obesity-related comorbidities, which includes hypertension and hypercholesterolemia.'''

result = gender_pipeline_sbert.annotate(text4)

result['class'][0]

'Female'

In [152]:
text5 = '''Prostate gland showing moderately differentiated infiltrating adenocarcinoma, 
Gleason 3 + 2 extending to the apex involving both lobes of the prostate, mainly right.'''

result = gender_pipeline_sbert.annotate(text5)

result['class'][0]

'Male'

In [154]:
text6 = '''SKIN: The patient has significant subcutaneous emphysema of the upper chest and 
anterior neck area although he states that the subcutaneous emphysema has improved significantly since yesterday.'''

result = gender_pipeline_sbert.annotate(text6)

result['class'][0]

'Male'

In [155]:
text7 = '''INDICATION: The patient is a 42-year-old XX who is five days out from transanal excision of a benign anterior base lesion.
xx presents today with diarrhea and bleeding. Digital exam reveals bright red blood on the finger.
xx is for exam under anesthesia and control of hemorrhage at this time.
'''
result = gender_pipeline_sbert.annotate(text7)

result['class'][0]

'Male'

In [156]:
text8 = '''INDICATION: ___ year old patient with complicated medical history of paraplegia
and chronic indwelling foley, recurrent MDR UTIs, hx Gallbladder fossa
abscess,type 2 DM, HTN, CAD, DVT s/p left AKA complicated complicated by
respiratory failure requiring tracheostomy and PEG placement, right ischium
osteomyelitis due to chronic pressure ulcers with acute shortness of breath...'''

result = gender_pipeline_sbert.annotate(text8)

result['class'][0]


'Male'

In [157]:

text9 = '''NDICATION: ___ year old xx with hx of polysubstance abuse, Hep C who
presented to OSH with N/V, BRBPR, found to have elevated lactate, acute renal
failure and with CT concerning acute pancreatitis, active colonic bleed (s/p
IR and now stable-appearing), proctocolitis, and shock, on vanc/zosyn and CRRT'''

result = gender_pipeline_sbert.annotate(text9)

result['class'][0]

'Male'

## Gender Classifier Pipeline with **biobert**

In [45]:
document = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

clf_tokenizer = Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")\

biobert_embeddings = BertEmbeddings().pretrained('biobert_pubmed_base_cased')\
    .setInputCols(["document",'token'])\
    .setOutputCol("bert_embeddings")

biobert_embeddings_avg = SentenceEmbeddings()\
    .setInputCols(["document", "bert_embeddings"])\
    .setOutputCol("sentence_bert_embeddings")\
    .setPoolingStrategy("AVERAGE")

genderClassifier = ClassifierDLModel.pretrained('classifierdl_gender_sbert', 'en', 'clinical/models')\
    .setInputCols(["document", "sentence_bert_embeddings"])\
    .setOutputCol("gender")
  
gender_pred_pipeline_bbert = Pipeline(stages = [
                                         documentAssembler,
                                         clf_tokenizer,
                                         biobert_embeddings,
                                         biobert_embeddings_avg,
                                         genderClassifier
                                         ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model_bbert = gender_pred_pipeline_bbert.fit(empty_data)




biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[OK!]
classifierdl_gender_sbert download started this may take some time.
Approximate size to download 22.4 MB
[OK!]


###  Sample Clinical Notes

In [159]:
text10 = '''social history: shows that  does not smoke cigarettes or drink alcohol,lives in a nursing home.family history: 
shows a family history of breast cancer.'''

gender_pipeline_bbert = LightPipeline(model_bbert)

result_bbert = gender_pipeline_bbert.annotate(text10)

result_bbert['gender'][0]

'Female'

In [160]:

text11 ='''Wand where xx was evaluated for xx smallbreasts as wellas dark areas on xx face and varicose veins to the back and
 posterior aspect of xx right lower extremity.'''

result_bbert = gender_pipeline_bbert.annotate(text11)

result_bbert['gender'][0]

'Female'

In [161]:

text12 = '''Prostate gland showing moderately differentiated infiltrating adenocarcinoma, 
Gleason 3 + 2 extending to the apex involving both lobes of the prostate, mainly right.'''

result_bbert = gender_pipeline_bbert.annotate(text12)

result_bbert['gender'][0]

'Male'